# 110 LangGraph: ReAct Agents with Tools and Reducers

**Workshop**: LangGraph 110
**Duration**: ~60 minutes
**Difficulty**: Advanced

## Learning Objectives

By completing this notebook, you will:
- Understand the ReAct (Reasoning and Acting) agent pattern
- Master the `add_messages` reducer for automatic state management
- Learn advanced type annotations: `Annotated` and `Sequence`
- Create custom tools for PAN-OS firewall operations
- Build a tool-calling agent that makes intelligent decisions
- Implement conditional edges for tool routing
- Understand multiple message types: `SystemMessage`, `ToolMessage`, `BaseMessage`

## Prerequisites

- **Knowledge**: Completed notebooks 108-109 (LLM Integration, Conversational Memory)
- **Understanding**: Graph patterns (103-107), manual state management (109)
- **Setup**: Anthropic API key configured

## What You'll Build

A **ReAct Agent** - an intelligent agent that can:
- Reason about which tool to use
- Act by calling appropriate tools
- Decide when to stop tool calling
- Handle multiple tools in a single query

**Example:**
```
Query: "Check if firewall fw-prod-01 needs an upgrade, then calculate the downtime"

Agent thinks: "I need to check version, then calculate downtime"
→ Calls: check_version(hostname="fw-prod-01")
→ Gets: "10.1.0"
→ Calls: calculate_downtime(current="10.1.0", target="11.0")
→ Gets: "45 minutes"
→ Responds: "Your firewall needs an upgrade. Estimated downtime: 45 minutes"
```

**Graph Structure:**
```
START → agent → tools → agent → tools → ... → END
         ↑         |
         └─────────┘
```

## Table of Contents

1. [Introduction to ReAct Agents](#1-introduction-to-react-agents)
2. [Setup and Understanding New Imports](#2-setup-and-understanding-new-imports)
3. [Understanding Reducers and add_messages](#3-understanding-reducers-and-add_messages)
4. [Creating PAN-OS Tools](#4-creating-panos-tools)
5. [Building the Agent State with Reducers](#5-building-the-agent-state-with-reducers)
6. [Creating the Agent Node](#6-creating-the-agent-node)
7. [Building the ReAct Graph](#7-building-the-react-graph)
8. [Testing the ReAct Agent](#8-testing-the-react-agent)
9. [Adding More Tools](#9-adding-more-tools)
10. [Summary](#10-summary)

---

## 1. Introduction to ReAct Agents

### What is a ReAct Agent?

**ReAct** stands for **Reasoning and Acting**. It's one of the most common and powerful agent patterns in LangGraph.

### How It Works

1. **Agent receives a query** → "Check if fw-prod-01 needs an upgrade"
2. **Agent reasons** → "I need to check the current version first"
3. **Agent acts** → Calls `check_version()` tool
4. **Tool returns result** → "Current version: 10.1.0"
5. **Agent reasons again** → "10.1.0 is old, needs upgrade to 11.0"
6. **Agent acts again** → Calls `get_upgrade_path()` tool
7. **Tool returns result** → "Path: 10.1 → 10.2 → 11.0"
8. **Agent finishes** → "Your firewall needs upgrade: 10.1 → 10.2 → 11.0"

### The Pattern

```
┌─────────┐
│  START  │
└────┬────┘
     ↓
┌────────────┐
│   Agent    │ ← Reasoning happens here
│  (LLM)     │
└─┬────────┬─┘
  │        │
  │   ┌────▼────┐
  │   │  Tools  │ ← Acting happens here
  │   │  Node   │
  │   └────┬────┘
  │        │
  └────────┘ (loops until done)
     ↓
┌────────────┐
│    END     │
└────────────┘
```

### Why ReAct Agents Are Powerful

**Problem with simple bots:**
- Can only answer from training data
- Can't perform actions
- Can't access external systems

**ReAct agents can:**
- Call APIs and tools
- Perform calculations
- Access databases
- Execute commands
- Make decisions based on results

### Real-World PAN-OS Use Cases

1. **Intelligent Upgrade Assistant**
   - Check current version
   - Calculate upgrade path
   - Estimate downtime
   - Verify compatibility

2. **Configuration Helper**
   - Validate NAT rules
   - Check security policies
   - Calculate subnet masks
   - Verify route tables

3. **Troubleshooting Bot**
   - Check system logs
   - Analyze traffic patterns
   - Verify interface status
   - Run diagnostic commands

### What Makes This Different from Notebook 105?

**Notebook 105:**
- Manual state management
- No tool calling
- LLM just talks
- Manual history synchronization

**Notebook 106:**
- Automatic state management (reducers!)
- Multiple tools available
- LLM can take actions
- Automatic history handling

Let's build it!

---

## 2. Setup and Understanding New Imports

### What's New in This Notebook?

We're introducing several powerful concepts:

1. **Annotated** - Advanced type hints with metadata
2. **Sequence** - Type annotation for ordered collections
3. **add_messages** - The magical reducer function
4. **BaseMessage** - Parent class for all message types
5. **SystemMessage** - Instructions for the LLM
6. **ToolMessage** - Results from tool execution
7. **@tool decorator** - Turns functions into LangGraph tools
8. **ToolNode** - Special node that manages all tools

### Import Breakdown

Let me explain each import in detail as we'll be using all of these!

In [ ]:
# Core typing imports - EXPANDED from previous notebooks
from typing import Annotated, Sequence, TypedDict

# LangChain message types - ALL THE MESSAGE TYPES!
from langchain_core.messages import (
    BaseMessage,      # Parent class for all messages
    HumanMessage,     # User to AI
    AIMessage,        # AI to User
    SystemMessage,    # Instructions to LLM
    ToolMessage,      # Tool results
)

# LangChain LLM integration
from langchain_anthropic import ChatAnthropic

# LangGraph core
from langgraph.graph import StateGraph, START, END

# LangGraph tool support - NEW!
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

# LangGraph reducer - THE GAME CHANGER!
from langgraph.graph.message import add_messages

# Visualization
from IPython.display import Image, display

# Environment variables
from dotenv import load_dotenv

print("✅ All imports successful!")
print("\n🆕 New imports in this notebook:")
print("  - Annotated: Advanced type hints with metadata")
print("  - Sequence: Type for ordered collections")
print("  - add_messages: Reducer function for automatic state management")
print("  - BaseMessage: Parent class for all message types")
print("  - SystemMessage: LLM instructions")
print("  - ToolMessage: Tool execution results")
print("  - @tool decorator: Creates LangGraph tools")
print("  - ToolNode: Manages all tools in a single node")

In [ ]:
# Load environment variables
load_dotenv()

print("✅ Environment loaded!")

### Understanding Annotated

**Annotated** adds metadata to type hints without changing the actual type.

**Why do we need it?** Sometimes the type alone isn't enough information.

**Example: Email Validation**

```python
# Without Annotated - just the type
email: str  # Could be ANY string!

# With Annotated - type + metadata
email: Annotated[str, "Must be valid email format"]
```

The actual type is still `str`, but now we have additional context!

Let's see it in action:

In [ ]:
# Demo: Annotated with metadata
FirewallHostname = Annotated[str, "Must be valid PAN-OS hostname format (e.g., fw-prod-01)"]

# Check the metadata
print("Type annotation: FirewallHostname")
print(f"Metadata: {FirewallHostname.__metadata__}")
print()

# Another example
PanosVersion = Annotated[str, "Must be in format X.Y.Z (e.g., 10.1.0)"]
print("Type annotation: PanosVersion")
print(f"Metadata: {PanosVersion.__metadata__}")
print()

print("💡 Key insight:")
print("   The actual type is still 'str'")
print("   But metadata provides additional context for validation/documentation")

### Understanding Message Type Hierarchy

LangGraph uses a **class hierarchy** for messages:

```
BaseMessage (Parent class)
    ├── HumanMessage (User input)
    ├── AIMessage (LLM response)
    ├── SystemMessage (LLM instructions)
    └── ToolMessage (Tool results)
```

**BaseMessage** is the foundational class - all message types inherit from it.

**Why does this matter?**
- We can use `Sequence[BaseMessage]` to accept ANY message type
- Simplifies our state definition
- All messages share common properties (content, id, etc.)

Let's explore the message types:

In [ ]:
# Create different message types
human_msg = HumanMessage(content="What's the upgrade path for PAN-OS 10.1?")
ai_msg = AIMessage(content="The path is: 10.1 → 10.2 → 11.0")
system_msg = SystemMessage(content="You are a PAN-OS expert assistant")
tool_msg = ToolMessage(content="Version check complete: 10.1.0", tool_call_id="123")

print("Message Types Demo:")
print("=" * 60)

messages = [human_msg, ai_msg, system_msg, tool_msg]

for msg in messages:
    print(f"\nType: {type(msg).__name__}")
    print(f"Content: {msg.content}")
    print(f"Is BaseMessage? {isinstance(msg, BaseMessage)}")
    
print("\n" + "=" * 60)
print("💡 Key insight:")
print("   ALL message types inherit from BaseMessage")
print("   This allows us to use Sequence[BaseMessage] for any message!")

---

## 3. Understanding Reducers and add_messages

### The Problem We're Solving

In notebook 109, we manually managed conversation history with tedious, error-prone code:

```python
# Manual approach from notebook 109
state["messages"].append(HumanMessage(content=user_input))
response = llm.invoke(state["messages"])
state["messages"].append(AIMessage(content=response.content))
conversation_history = result["messages"]  # Manual sync!
```

**Problems with this approach:**
- Need to manually `.append()` in every node
- Easy to forget and lose conversation context
- Must manually synchronize `conversation_history`
- More code = more bugs
- Repetitive boilerplate in every node

### What is a Reducer?

**A reducer is a function that controls HOW state updates are merged into existing state.**

Think of it like merge strategies in Git:
- **Without reducer**: Replace (like `git reset --hard`)
- **With reducer**: Merge intelligently (like `git merge`)

**Default behavior (no reducer):**
```python
state = {"messages": [msg1, msg2]}
update = {"messages": [msg3]}
# Result: {"messages": [msg3]}  # ❌ OVERWRITES! Lost msg1 and msg2!
```

**With add_messages reducer:**
```python
state = {"messages": [msg1, msg2]}
update = {"messages": [msg3]}
# Result: {"messages": [msg1, msg2, msg3]}  # ✅ APPENDS! Keeps history!
```

### How add_messages Works

The `add_messages` reducer:
1. Takes the existing messages list
2. Takes the new messages from the update
3. **Intelligently merges** them (appends new messages)
4. Returns the combined list

**Visual representation:**

```
State Before Update:
┌─────────────────────────────────┐
│ messages: [                     │
│   HumanMessage("Hello"),        │
│   AIMessage("Hi there!")        │
│ ]                               │
└─────────────────────────────────┘

Update from Node:
┌─────────────────────────────────┐
│ return {                        │
│   "messages": [                 │
│     HumanMessage("How are you?")│
│   ]                             │
│ }                               │
└─────────────────────────────────┘

WITHOUT Reducer (default):           WITH add_messages Reducer:
┌─────────────────────────────────┐  ┌─────────────────────────────────┐
│ messages: [                     │  │ messages: [                     │
│   HumanMessage("How are you?")  │  │   HumanMessage("Hello"),        │
│ ]                               │  │   AIMessage("Hi there!"),       │
│                                 │  │   HumanMessage("How are you?")  │
│ ❌ LOST CONVERSATION HISTORY!   │  │ ]                               │
└─────────────────────────────────┘  │ ✅ PRESERVED EVERYTHING!        │
                                      └─────────────────────────────────┘
```

### The Magic Formula

```python
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    #         ^^^^^^^^^ ^^^^^^^^^^ ^^^^^^^^^^^^ ^^^^^^^^^^^^
    #         │         │          │            │
    #         │         │          │            └─ Reducer function
    #         │         │          └─ Type (any BaseMessage subclass)
    #         │         └─ Ordered collection interface
    #         └─ Metadata container (holds type + reducer)
```

### Breaking Down Each Component

**1. Annotated** - Metadata container
```python
from typing import Annotated

# Annotated[ActualType, metadata1, metadata2, ...]
# The actual type is the first argument
# Everything else is metadata
```

**2. Sequence[BaseMessage]** - Ordered collection of any message type
```python
from typing import Sequence
from langchain_core.messages import BaseMessage

# Sequence is an abstract interface for ordered collections
# In practice, the runtime value will be a list
# But we use Sequence to accept any ordered collection
# BaseMessage means: HumanMessage, AIMessage, SystemMessage, ToolMessage, etc.
```

**3. add_messages** - The reducer that implements append behavior
```python
from langgraph.graph.message import add_messages

# This function is called automatically whenever state["messages"] is updated
# It receives: (existing_messages, new_messages)
# It returns: existing_messages + new_messages (append behavior)
```

### Code Comparison: Before vs After

**Before (Notebook 109) - Manual State Management:**

```python
from typing import TypedDict, Union, List
from langchain_core.messages import HumanMessage, AIMessage

class ConversationState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]  # No reducer!

def chat_node(state: ConversationState) -> ConversationState:
    """Every node must manually manage state."""
    messages = state["messages"]
    
    # Manual append for user message
    user_message = HumanMessage(content="Hello")
    messages.append(user_message)  # ⚠️ Manual!
    
    # Get response
    response = llm.invoke(messages)
    
    # Manual append for AI message  
    messages.append(AIMessage(content=response.content))  # ⚠️ Manual!
    
    # Manual state update
    return {"messages": messages}  # ⚠️ Must return full list!

# After graph runs
conversation_history = result["messages"]  # ⚠️ Manual synchronization!
```

**After (Notebook 110) - Automatic State Management:**

```python
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]  # ✅ Reducer!

def chat_node(state: AgentState) -> AgentState:
    """Reducer handles everything automatically."""
    messages = state["messages"]
    
    # Get response
    response = llm.invoke(messages)
    
    # Just return new messages - reducer handles merging!
    return {"messages": [response]}  # ✅ That's it!
    # No manual append!
    # No manual state management!
    # Reducer automatically appends to existing messages!

# After graph runs - messages are automatically maintained!
# No manual synchronization needed!
```

**Lines of code comparison:**
- Manual approach: ~15 lines per node
- Reducer approach: ~5 lines per node
- **Reduction: 66% less code!**

### Why This Matters for ReAct Agents

In a ReAct agent, messages flow constantly:
1. User sends `HumanMessage`
2. Agent responds with `AIMessage` (may include tool_calls)
3. Tools respond with `ToolMessage`
4. Agent responds with another `AIMessage`
5. Repeat...

**Without reducer:**
```python
def agent_node(state):
    messages = state["messages"]
    response = llm.invoke(messages)
    messages.append(response)  # Manual!
    return {"messages": messages}

def tool_node(state):
    messages = state["messages"]
    # Execute tool
    tool_message = ToolMessage(content=result)
    messages.append(tool_message)  # Manual!
    return {"messages": messages}

# Every. Single. Node. Must. Append. Manually. 😫
```

**With reducer:**
```python
def agent_node(state):
    response = llm.invoke(state["messages"])
    return {"messages": [response]}  # Automatic!

def tool_node(state):
    # Execute tool
    tool_message = ToolMessage(content=result)
    return {"messages": [tool_message]}  # Automatic!

# Clean, simple, no manual append! 🎉
```

### Advanced: How add_messages Handles Updates

The `add_messages` reducer is smart about message types:

```python
# Example: Updating an existing message by ID
state = {
    "messages": [
        AIMessage(content="Old", id="msg-123")
    ]
}

# If you return a message with the same ID, it UPDATES instead of appending
update = {
    "messages": [
        AIMessage(content="New", id="msg-123")  # Same ID!
    ]
}

# Result: {"messages": [AIMessage(content="New", id="msg-123")]}
# The reducer detected the matching ID and updated instead of appending!
```

This is useful for:
- Updating streaming responses
- Correcting mistakes
- Replacing temporary messages

### Let's See It In Action

Let me demonstrate the difference with a side-by-side comparison:

In [ ]:
# Demo: Reducer behavior - simulating what happens behind the scenes

print("SCENARIO: Building a conversation history across multiple nodes")
print("="*80)
print()

print("WITHOUT REDUCER (default TypedDict behavior):")
print("-"*80)

# Simulating default behavior (replace/overwrite)
print("Initial state after START:")
state_without = {"messages": [HumanMessage(content="Hello")]}
print(f"  messages: {[m.content for m in state_without['messages']]}")
print()

print("Node 1 returns: {'messages': [AIMessage('Hi there!')]}")
# Default behavior: REPLACE
state_without = {"messages": [AIMessage(content="Hi there!")]}
print(f"  State after Node 1: {[m.content for m in state_without['messages']]}")
print("  ❌ LOST 'Hello'! Only has the new message.")
print()

print("Node 2 returns: {'messages': [HumanMessage('How are you?')]}")
# Default behavior: REPLACE again
state_without = {"messages": [HumanMessage(content="How are you?")]}
print(f"  State after Node 2: {[m.content for m in state_without['messages']]}")
print("  ❌ LOST EVERYTHING! Conversation context is gone!")
print()
print()

print("WITH add_messages REDUCER:")
print("-"*80)

# Simulating reducer behavior (append)
print("Initial state after START:")
state_with = {"messages": [HumanMessage(content="Hello")]}
print(f"  messages: {[m.content for m in state_with['messages']]}")
print()

print("Node 1 returns: {'messages': [AIMessage('Hi there!')]}")
# Reducer behavior: APPEND
state_with["messages"].append(AIMessage(content="Hi there!"))
print(f"  State after Node 1: {[m.content for m in state_with['messages']]}")
print("  ✅ KEPT 'Hello' AND added 'Hi there!'")
print()

print("Node 2 returns: {'messages': [HumanMessage('How are you?')]}")
# Reducer behavior: APPEND again  
state_with["messages"].append(HumanMessage(content="How are you?"))
print(f"  State after Node 2: {[m.content for m in state_with['messages']]}")
print("  ✅ KEPT EVERYTHING! Full conversation preserved!")
print()

print("Node 3 returns: {'messages': [AIMessage('I am doing great!')]}")
state_with["messages"].append(AIMessage(content="I am doing great!"))
print(f"  State after Node 3: {[m.content for m in state_with['messages']]}")
print("  ✅ Complete conversation history maintained automatically!")
print()
print()

print("="*80)
print("COMPARISON SUMMARY:")
print("="*80)
print()
print("Without Reducer:")
print("  - Each node REPLACES the entire messages list")
print("  - Conversation history is LOST")
print("  - Must manually pass full list from node to node")
print("  - Easy to make mistakes")
print()
print("With add_messages Reducer:")
print("  - Each node's messages are APPENDED automatically")
print("  - Conversation history is PRESERVED")
print("  - Nodes only return NEW messages")
print("  - Simple, clean, automatic!")
print()
print("💡 Key Insight:")
print("   The reducer acts like a smart merge function:")
print("   new_state = reducer(old_state, node_update)")
print("   new_state = old_state + node_update  # Append behavior!")
print()
print("💡 In your nodes, you can now just:")
print("   return {'messages': [new_message]}  # That's it!")
print("   # The reducer handles appending to existing messages!")

---

## 4. Creating SCM Tools with StructuredTool

### What Are Tools?

**Tools** are functions that the LLM can call to perform actions or retrieve information from Strata Cloud Manager.

**Examples:**
- Fetch address objects from SCM
- Create new address objects
- List addresses with filtering
- Create address groups

### StructuredTool.from_function() Pattern

In notebooks, we use `StructuredTool.from_function()` instead of the `@tool` decorator to avoid re-execution issues:

```python
def _my_tool_function(param: str) -> str:
    """This docstring is CRITICAL! It tells the LLM what the tool does."""
    return f"Result: {param}"

my_tool = StructuredTool.from_function(
    func=_my_tool_function,
    name="my_tool",
    description="Clear description for the LLM"
)
```

### Why StructuredTool for Notebooks?

**Problem with @tool decorator:**
- Re-running cells causes decorator conflicts
- Can lead to duplicate tool registrations
- Harder to debug in notebook environments

**Benefits of StructuredTool:**
- Clean separation of function and tool metadata
- No decorator conflicts on re-execution
- Explicit control over name and description
- Better for iterative development

### Why Docstrings Are Required

The LLM uses the docstring and description to decide:
1. **When** to use the tool
2. **What** parameters to pass
3. **How** to interpret results

**Without good descriptions, the tool won't work effectively!**

### Our First Tool: Fetch Address Object

Let's create a tool to fetch address objects from SCM (simulated):

In [ ]:
# Import StructuredTool
from langchain_core.tools import StructuredTool
import uuid

# Define the function first (prefix with _ to indicate it's internal)
def _fetch_address_object(name: str, folder: str) -> str:
    """
    Fetch an address object from Strata Cloud Manager.
    
    Args:
        name: Name of the address object (e.g., 'web_server_01')
        folder: SCM folder where the object is stored (e.g., 'Texas')
    
    Returns:
        JSON representation of the address object with its configuration
    
    Use this tool when the user asks about existing address objects or wants
    to check if an address object exists before creating it.
    """
    # Simulated SCM database of address objects
    scm_addresses = {
        ("web_server_01", "Texas"): {
            "id": "addr-12345678",
            "name": "web_server_01",
            "ip_netmask": "10.0.1.10/32",
            "folder": "Texas",
            "description": "Production web server",
            "tag": ["Production", "Web"]
        },
        ("db_server_01", "Texas"): {
            "id": "addr-87654321",
            "name": "db_server_01",
            "ip_netmask": "10.0.2.10/32",
            "folder": "Texas",
            "description": "Production database server",
            "tag": ["Production", "Database"]
        },
        ("internal_network", "Texas"): {
            "id": "addr-11111111",
            "name": "internal_network",
            "ip_netmask": "192.168.1.0/24",
            "folder": "Texas",
            "description": "Internal network segment",
            "tag": ["Network", "Internal"]
        }
    }
    
    # Look up the address object
    key = (name, folder)
    if key in scm_addresses:
        addr = scm_addresses[key]
        return f"Found address object: {addr['name']}\n" \
               f"ID: {addr['id']}\n" \
               f"IP/Netmask: {addr['ip_netmask']}\n" \
               f"Folder: {addr['folder']}\n" \
               f"Description: {addr['description']}\n" \
               f"Tags: {', '.join(addr['tag'])}"
    else:
        return f"Address object '{name}' not found in folder '{folder}'"

# Create the tool using StructuredTool.from_function()
fetch_address_object = StructuredTool.from_function(
    func=_fetch_address_object,
    name="fetch_address_object",
    description="Fetch an address object from Strata Cloud Manager by name and folder. "
                "Use this when the user asks about existing address objects or wants to "
                "check if an object exists."
)

print("✅ Tool created: fetch_address_object")
print("\n💡 Notice the StructuredTool.from_function() pattern:")
print("   1. Define function with _ prefix (_fetch_address_object)")
print("   2. Comprehensive docstring in the function")
print("   3. Create tool with StructuredTool.from_function()")
print("   4. Explicit name and description")
print("   5. No decorator conflicts on re-execution!")
print("\n💡 This simulates the SCM API pattern:")
print("   client.address.fetch(name='web_server_01', folder='Texas')")

### Testing the Tool Manually

Before using tools in an agent, let's test them directly to ensure they work correctly:

In [ ]:
# Test the tool directly
print("Testing fetch_address_object tool:")
print("=" * 60)

test_queries = [
    ("web_server_01", "Texas"),
    ("internal_network", "Texas"),
    ("nonexistent", "Texas")
]

for name, folder in test_queries:
    result = fetch_address_object.invoke({"name": name, "folder": folder})
    print(f"\nQuery: name='{name}', folder='{folder}'")
    print(f"Result:\n{result}")
    print("-" * 60)

print("\n" + "=" * 60)
print("✅ Tool works! Now let's add more SCM tools.")

### Adding More SCM Tools

Now let's create a comprehensive suite of SCM tools that demonstrate real Strata Cloud Manager operations:

1. **create_address_object** - Create new address objects with validation
2. **list_addresses** - List and filter address objects
3. **create_address_group** - Create address groups from existing objects

These tools simulate the pan-scm-sdk API patterns you'll use in production.

In [ ]:
# Simulated SCM database (shared across tools)
# In production, this would be managed by the SCM API
scm_database = {
    "addresses": {
        ("web_server_01", "Texas"): {
            "id": "addr-12345678",
            "name": "web_server_01",
            "ip_netmask": "10.0.1.10/32",
            "folder": "Texas",
            "description": "Production web server",
            "tag": ["Production", "Web"]
        },
        ("db_server_01", "Texas"): {
            "id": "addr-87654321",
            "name": "db_server_01",
            "ip_netmask": "10.0.2.10/32",
            "folder": "Texas",
            "description": "Production database server",
            "tag": ["Production", "Database"]
        },
        ("internal_network", "Texas"): {
            "id": "addr-11111111",
            "name": "internal_network",
            "ip_netmask": "192.168.1.0/24",
            "folder": "Texas",
            "description": "Internal network segment",
            "tag": ["Network", "Internal"]
        }
    },
    "address_groups": {}
}

# Tool 1: create_address_object
def _create_address_object(name: str, ip_netmask: str, folder: str, description: str = "") -> str:
    """
    Create a new address object in Strata Cloud Manager.
    
    Args:
        name: Name of the address object (e.g., 'web_server_02')
        ip_netmask: IP address with CIDR notation (e.g., '10.0.1.20/32' or '192.168.1.0/24')
        folder: SCM folder where the object will be created (e.g., 'Texas')
        description: Optional description of the address object
    
    Returns:
        Confirmation message with the created object details including its ID
    
    Use this tool when the user wants to create a new address object in SCM.
    Validate that the object doesn't already exist before creating.
    """
    import re
    
    # Validate IP/netmask format
    ip_pattern = r'^(\d{1,3}\.){3}\d{1,3}/\d{1,2}$'
    if not re.match(ip_pattern, ip_netmask):
        return f"❌ Error: Invalid IP/netmask format: '{ip_netmask}'. Expected format: '10.0.1.10/32'"
    
    # Check if object already exists
    key = (name, folder)
    if key in scm_database["addresses"]:
        existing = scm_database["addresses"][key]
        return f"❌ Error: Address object '{name}' already exists in folder '{folder}'\n" \
               f"Existing object ID: {existing['id']}\n" \
               f"IP/Netmask: {existing['ip_netmask']}\n" \
               f"Use fetch_address_object to view it or choose a different name."
    
    # Create the address object
    object_id = f"addr-{uuid.uuid4().hex[:8]}"
    new_address = {
        "id": object_id,
        "name": name,
        "ip_netmask": ip_netmask,
        "folder": folder,
        "description": description,
        "tag": []
    }
    
    scm_database["addresses"][key] = new_address
    
    return f"✅ Successfully created address object!\n" \
           f"Name: {name}\n" \
           f"ID: {object_id}\n" \
           f"IP/Netmask: {ip_netmask}\n" \
           f"Folder: {folder}\n" \
           f"Description: {description}\n" \
           f"\n💡 This simulates: client.address.create({{'name': '{name}', 'ip_netmask': '{ip_netmask}', 'folder': '{folder}'}})"

create_address_object = StructuredTool.from_function(
    func=_create_address_object,
    name="create_address_object",
    description="Create a new address object in Strata Cloud Manager with IP/netmask. "
                "Use this when the user wants to create address objects. "
                "Always validate the object doesn't exist first using fetch_address_object."
)

# Tool 2: list_addresses
def _list_addresses(folder: str, tag_filter: str = "") -> str:
    """
    List address objects in a specified folder with optional tag filtering.
    
    Args:
        folder: SCM folder to list addresses from (e.g., 'Texas')
        tag_filter: Optional tag to filter by (e.g., 'Production', 'Web')
    
    Returns:
        List of address objects matching the filter criteria
    
    Use this tool when the user wants to see all address objects in a folder
    or filter address objects by tags.
    """
    addresses = []
    
    for (name, obj_folder), obj in scm_database["addresses"].items():
        if obj_folder == folder:
            # Apply tag filter if specified
            if tag_filter and tag_filter not in obj.get("tag", []):
                continue
            addresses.append(obj)
    
    if not addresses:
        filter_msg = f" with tag '{tag_filter}'" if tag_filter else ""
        return f"No address objects found in folder '{folder}'{filter_msg}"
    
    result = f"Found {len(addresses)} address object(s) in folder '{folder}'"
    if tag_filter:
        result += f" with tag '{tag_filter}'"
    result += ":\n\n"
    
    for addr in addresses:
        result += f"- {addr['name']}\n"
        result += f"  ID: {addr['id']}\n"
        result += f"  IP/Netmask: {addr['ip_netmask']}\n"
        if addr['description']:
            result += f"  Description: {addr['description']}\n"
        if addr['tag']:
            result += f"  Tags: {', '.join(addr['tag'])}\n"
        result += "\n"
    
    result += f"💡 This simulates: client.address.list(folder='{folder}'"
    if tag_filter:
        result += f", tags=['{tag_filter}']"
    result += ")"
    
    return result

list_addresses = StructuredTool.from_function(
    func=_list_addresses,
    name="list_addresses",
    description="List address objects in an SCM folder with optional tag filtering. "
                "Use this when the user wants to see what address objects exist in a folder "
                "or filter addresses by tags."
)

# Tool 3: create_address_group
def _create_address_group(name: str, members: str, folder: str, description: str = "") -> str:
    """
    Create an address group in Strata Cloud Manager.
    
    Args:
        name: Name of the address group (e.g., 'web_servers')
        members: Comma-separated list of address object names (e.g., 'web_server_01,web_server_02')
        folder: SCM folder where the group will be created (e.g., 'Texas')
        description: Optional description of the address group
    
    Returns:
        Confirmation message with the created group details
    
    Use this tool when the user wants to create an address group from existing address objects.
    Always verify that the member address objects exist before creating the group.
    """
    # Parse members
    member_list = [m.strip() for m in members.split(",")]
    
    # Validate that all members exist
    missing_members = []
    for member in member_list:
        key = (member, folder)
        if key not in scm_database["addresses"]:
            missing_members.append(member)
    
    if missing_members:
        return f"❌ Error: Cannot create address group. The following address objects don't exist:\n" \
               f"  {', '.join(missing_members)}\n" \
               f"Create these address objects first using create_address_object."
    
    # Check if group already exists
    key = (name, folder)
    if key in scm_database["address_groups"]:
        existing = scm_database["address_groups"][key]
        return f"❌ Error: Address group '{name}' already exists in folder '{folder}'\n" \
               f"Existing group ID: {existing['id']}\n" \
               f"Members: {', '.join(existing['static'])}"
    
    # Create the address group
    group_id = f"agrp-{uuid.uuid4().hex[:8]}"
    new_group = {
        "id": group_id,
        "name": name,
        "static": member_list,
        "folder": folder,
        "description": description,
        "tag": []
    }
    
    scm_database["address_groups"][key] = new_group
    
    return f"✅ Successfully created address group!\n" \
           f"Name: {name}\n" \
           f"ID: {group_id}\n" \
           f"Members: {', '.join(member_list)} ({len(member_list)} objects)\n" \
           f"Folder: {folder}\n" \
           f"Description: {description}\n" \
           f"\n💡 This simulates: client.address_group.create({{'name': '{name}', 'static': {member_list}, 'folder': '{folder}'}})"

create_address_group = StructuredTool.from_function(
    func=_create_address_group,
    name="create_address_group",
    description="Create an address group in Strata Cloud Manager from existing address objects. "
                "Provide members as comma-separated names. "
                "Use this when the user wants to group multiple address objects together."
)

print("✅ Comprehensive SCM tool suite created!")
print("\nAvailable tools:")
print("  1. fetch_address_object - Fetch existing address objects")
print("  2. create_address_object - Create new address objects with validation")
print("  3. list_addresses - List and filter address objects")
print("  4. create_address_group - Create address groups from existing objects")
print("\n💡 These tools simulate real pan-scm-sdk API patterns!")
print("   - Input validation")
print("   - Existence checks")
print("   - Error handling")
print("   - Realistic responses")

---

## 5. Building the Agent State with Reducers

### The New State Pattern

Remember the evolution:

**Notebook 104:** `messages: List[HumanMessage]`
**Notebook 105:** `messages: List[Union[HumanMessage, AIMessage]]`
**Notebook 106:** `messages: Annotated[Sequence[BaseMessage], add_messages]` ⭐

### Why This is Better

1. **Sequence[BaseMessage]** - Accepts ALL message types (Human, AI, System, Tool)
2. **add_messages** - Automatically handles appending
3. **No manual state manipulation** - Just return new messages!

### The State Definition

This single line replaces all the manual work from notebook 105:

In [ ]:
class AgentState(TypedDict):
    """State for our ReAct agent with automatic message management."""
    messages: Annotated[Sequence[BaseMessage], add_messages]
    #         ^^^^^^^^^           ^^^^^^^^^^^^^  ^^^^^^^^^^^^
    #         │                   │              │
    #         │                   │              └─ Reducer: auto-append
    #         │                   └─ Type: any message type
    #         └─ Metadata container

print("✅ Agent state defined with reducer!")
print("\nState breakdown:")
print("  - Annotated: Provides metadata")
print("  - Sequence[BaseMessage]: Ordered collection of any message type")
print("  - add_messages: Reducer that automatically appends new messages")
print("\n💡 This ONE line does all the work we did manually in notebook 105!")
print("   No more manual .append()")
print("   No more conversation_history variable")
print("   No more manual synchronization!")

---

## 6. Creating the Agent Node

### Setting Up the LLM with Tools

First, we need to:
1. Create the LLM
2. Create a list of available tools
3. Bind the tools to the LLM using `.bind_tools()`

This tells the LLM: "These are the tools you can use!"

In [ ]:
# Create the list of SCM tools
tools = [
    fetch_address_object,
    create_address_object,
    list_addresses,
    create_address_group
]

# Initialize LLM
llm = ChatAnthropic(model="claude-3-5-sonnet-20241022", temperature=0)

# Bind tools to LLM
model_with_tools = llm.bind_tools(tools)

print("✅ LLM initialized and bound with SCM tools!")
print(f"\nAvailable tools: {len(tools)}")
for i, tool in enumerate(tools, 1):
    print(f"  {i}. {tool.name}")
print("\n💡 The LLM now knows it can call these SCM tools to manage address objects!")

### Creating the Agent Node Function

This node:
1. Adds a system message with SCM-specific instructions
2. Invokes the LLM with full conversation history
3. Returns the response (reducer handles appending!)

**Key difference from notebook 109:** We use `SystemMessage` for clearer LLM instructions about SCM operations.

In [ ]:
def call_model(state: AgentState) -> AgentState:
    """
    The agent node that processes queries and decides on tool usage.
    
    This node:
    1. Receives the current conversation state
    2. Adds system instructions for SCM operations
    3. Invokes the LLM (which may call tools)
    4. Returns the response
    
    The reducer handles appending the response automatically!
    """
    # Define system instructions for SCM operations
    system_prompt = SystemMessage(
        content="You are a Strata Cloud Manager (SCM) automation assistant. "
                "You help users manage address objects and address groups in SCM. "
                "Use the available tools to fetch, create, list, and group address objects. "
                "Always validate that objects exist before referencing them. "
                "When creating multiple objects, do them one at a time and verify success. "
                "Be helpful and explain what you're doing at each step."
    )
    
    # Get current messages from state
    messages = state["messages"]
    
    # Invoke LLM with system message + conversation history
    response = model_with_tools.invoke([system_prompt] + messages)
    
    # Return response - add_messages reducer will append it!
    return {"messages": [response]}

print("✅ Agent node function defined with SCM-specific instructions!")
print("\n💡 Key differences from notebook 109:")
print("   1. SystemMessage with SCM-specific guidance")
print("   2. No manual .append() - reducer handles it!")
print("   3. LLM can now call SCM tools")
print("   4. Simpler return: just {'messages': [response]}")

---

## 7. Building the ReAct Graph

### The ReAct Pattern Requires

1. **Agent node** - Makes decisions and calls tools
2. **Tool node** - Executes the tools
3. **Conditional edge** - Decides: continue to tools OR end?
4. **Loop back edge** - Returns from tools to agent

### The Decision Logic

After the agent runs, we need to check:
- **Does the response contain tool calls?** → Go to tools
- **No tool calls?** → We're done, go to END

This creates the reasoning-acting loop!

In [ ]:
def should_continue(state: AgentState) -> str:
    """
    Conditional edge function that determines next step.
    
    Checks the last message:
    - If it contains tool_calls → route to "tools"
    - If no tool_calls → route to END
    
    This creates the ReAct loop!
    """
    messages = state["messages"]
    last_message = messages[-1]
    
    # Check if the last message has tool calls
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    else:
        return END

print("✅ Conditional edge function defined!")
print("\nDecision logic:")
print("  - If last message has tool_calls → route to 'tools'")
print("  - If no tool_calls → route to END")
print("\n💡 This creates the loop:")
print("   agent → tools → agent → tools → ... → END")

### Building the Graph

Now we assemble all the pieces:

In [ ]:
# Initialize the graph
graph = StateGraph(AgentState)

# Add the agent node
graph.add_node("agent", call_model)

# Add the tool node - ToolNode automatically handles all tools in the list!
tool_node = ToolNode(tools)
graph.add_node("tools", tool_node)

# Set entry point
graph.set_entry_point("agent")

# Add conditional edge from agent
graph.add_conditional_edges(
    "agent",           # Source node
    should_continue,   # Decision function
    {
        "tools": "tools",  # If should_continue returns "tools", go to tools node
        END: END           # If should_continue returns END, finish
    }
)

# Add edge from tools back to agent (creates the loop!)
graph.add_edge("tools", "agent")

# Compile the graph
app = graph.compile()

print("✅ ReAct graph compiled!")
print("\nGraph structure:")
print("  START → agent")
print("           ↓  ↑")
print("         tools")
print("           ↓")
print("         END")
print("\n💡 The conditional edge + loop create the ReAct pattern!")

In [ ]:
# Visualize the graph
display(Image(app.get_graph().draw_mermaid_png()))

print("\n📊 Graph visualization shows:")
print("  - agent node (reasoning)")
print("  - tools node (acting)")
print("  - Conditional routing from agent")
print("  - Loop from tools back to agent")

---

## 8. Testing the ReAct Agent with SCM Operations

### Test 1: Simple Tool Call - Fetch Address Object

Let's start with a basic query that requires fetching an existing address object:

In [ ]:
# Helper function to display results nicely
def print_stream(stream):
    """Print the stream output in a readable format."""
    for event in stream:
        for node_name, data in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if "messages" in data:
                for msg in data["messages"]:
                    msg_type = type(msg).__name__
                    print(f"\n[{msg_type}]")
                    
                    if hasattr(msg, "content"):
                        print(f"Content: {msg.content}")
                    
                    if hasattr(msg, "tool_calls") and msg.tool_calls:
                        print(f"Tool Calls: {len(msg.tool_calls)}")
                        for tc in msg.tool_calls:
                            print(f"  - {tc['name']}({tc['args']})")

print("✅ Helper function defined for pretty output")

In [ ]:
# Test 1: Simple tool call - fetch address object
print("TEST 1: Fetch an existing address object")
print("="*60)

query = "What is the IP address of web_server_01 in the Texas folder?"
stream = app.stream({"messages": [HumanMessage(content=query)]})

print_stream(stream)

print("\n" + "="*60)
print("✅ SUCCESS! The agent:")
print("   1. Received the query about an address object")
print("   2. Decided to call fetch_address_object tool")
print("   3. Got the result (10.0.1.10/32)")
print("   4. Formatted a nice response with the IP address")
print("\n💡 This is the ReAct pattern in action with SCM!")
print("   Reason: 'I need to fetch this address object'")
print("   Act: Calls fetch_address_object(name='web_server_01', folder='Texas')")
print("   Observe: Gets the result and responds")

### Test 2: Query Without Tool

What happens if the agent doesn't need to use a tool?

In [ ]:
# Test 2: No tool needed - general question
print("\nTEST 2: General question (no tool needed)")
print("="*60)

query = "What is Strata Cloud Manager?"
stream = app.stream({"messages": [HumanMessage(content=query)]})

print_stream(stream)

print("\n" + "="*60)
print("✅ ROBUST! The agent:")
print("   1. Received a general question about SCM")
print("   2. Determined no tool was needed")
print("   3. Answered from its training data")
print("   4. Went directly to END")
print("\n💡 The conditional edge correctly routed to END!")
print("   The agent is smart enough to know when NOT to use tools")

---

## 9. Autonomous Multi-Step Workflows

### The Power of ReAct: Chaining Tool Calls

The ReAct pattern truly shines when the agent needs to perform multiple operations autonomously. Let's demonstrate a complex workflow where the agent must:

1. **Create multiple address objects** (4 web servers)
2. **Verify each creation succeeded**
3. **Create an address group** containing all 4 servers
4. **Handle any errors** that occur along the way

This simulates a real-world SCM automation task: "Set up 4 new web servers and group them for a security policy."

### Test 3: Autonomous Workflow - Create and Group Servers

Watch how the agent breaks down a complex task into multiple tool calls:

---

## 10. Practice Exercises

Now that you've mastered the ReAct pattern with SCM tools, try these challenges to deepen your understanding:

### Exercise 1: Add Security Policy Tools (Intermediate)

Create tools for managing security policies:

```python
def _create_security_rule(name: str, source_zone: str, dest_zone: str, 
                          action: str, folder: str) -> str:
    """Create a security policy rule."""
    # Your implementation here
    pass

def _list_security_rules(folder: str, action_filter: str = "") -> str:
    """List security policy rules with optional filtering."""
    # Your implementation here
    pass
```

**Requirements:**
- Use `StructuredTool.from_function()` pattern
- Include input validation (valid zones, actions)
- Return clear error messages
- Test with the agent

### Exercise 2: Add Update Address Object Tool (Intermediate)

Create a tool to update existing address objects:

```python
def _update_address_object(name: str, folder: str, 
                           new_ip_netmask: str = "", 
                           new_description: str = "") -> str:
    """Update an existing address object."""
    # Your implementation here
    # Hint: Fetch first, validate changes, then update
    pass
```

**Requirements:**
- Fetch the existing object first
- Validate it exists before updating
- Allow partial updates (only change specified fields)
- Return before/after comparison

### Exercise 3: Complex Autonomous Workflow (Advanced)

Create a workflow that:
1. Lists all address objects with tag "Production"
2. Creates a new address group from those objects
3. Creates a security rule allowing traffic from "trust" to "dmz" using that group

Test with: *"Create a security rule for all production servers allowing traffic from trust to dmz zone"*

**Challenge:** The agent must chain 3+ tools autonomously!

### Exercise 4: Add Retry Logic (Advanced)

Enhance the `create_address_object` tool with automatic retry for name conflicts:

```python
def _create_address_object_with_retry(name: str, ip_netmask: str, 
                                      folder: str, description: str = "") -> str:
    """
    Create address object with automatic name conflict resolution.
    If name exists, try appending _2, _3, etc.
    """
    # Your implementation here
    pass
```

**Requirements:**
- Try original name first
- If conflict, append suffix: `_2`, `_3`, etc.
- Max 5 attempts
- Return final name used

### Exercise 5: Build a Tool Execution Dashboard (Advanced)

Create a function that tracks and displays tool execution statistics:

```python
tool_stats = {
    "total_calls": 0,
    "by_tool": {},
    "errors": 0,
    "success": 0
}

def track_tool_execution(tool_name: str, success: bool):
    """Track tool execution statistics."""
    # Your implementation
    pass

def display_tool_dashboard():
    """Display tool usage statistics."""
    # Your implementation
    pass
```

Integrate this with the agent to see which tools are most used!

### Solutions

Solutions for these exercises are available in the workshop repository under `solutions/110_exercises.ipynb`.

---

## 11. Summary

Congratulations! You've built a fully functional ReAct agent with comprehensive SCM tool calling capabilities. This is a major milestone in your LangGraph journey!

In [ ]:
# This cell is no longer needed - we built the comprehensive tool suite earlier
# The agent is already configured with all 4 SCM tools:
# - fetch_address_object
# - create_address_object  
# - list_addresses
# - create_address_group

print("✅ Agent is ready with comprehensive SCM tool suite!")
print("\nTo recap, our agent has:")
print("  1. fetch_address_object - Retrieve address objects")
print("  2. create_address_object - Create new address objects with validation")
print("  3. list_addresses - List and filter address objects")
print("  4. create_address_group - Group address objects")
print("\n💡 The agent can autonomously chain these tools together!")

In [ ]:
# Test 3: Autonomous multi-step workflow - Create 4 servers and group them
print("\nTEST 3: Autonomous workflow - Create 4 web servers and group them")
print("="*60)

query = """Create 4 new web servers in the Texas folder:
- web_server_02 at 10.0.1.20/32
- web_server_03 at 10.0.1.30/32
- web_server_04 at 10.0.1.40/32
- web_server_05 at 10.0.1.50/32

Then create an address group called 'new_web_servers' containing all 4 servers."""

stream = app.stream({"messages": [HumanMessage(content=query)]})

print_stream(stream)

print("\n" + "="*60)
print("✅ INCREDIBLE! The agent autonomously:")
print("   1. Understood it needed to create 4 separate address objects")
print("   2. Called create_address_object 4 times (once for each server)")
print("   3. Verified each creation succeeded")
print("   4. Called create_address_group with all 4 server names")
print("   5. Confirmed the group was created successfully")
print("   6. Synthesized a complete summary for the user")
print("\n💡 This is the full power of the ReAct pattern!")
print("   - Autonomous multi-step decision making")
print("   - Sequential tool execution")
print("   - Error awareness and recovery")
print("   - Result synthesis and summarization")
print("\n💡 Tool calling sequence:")
print("   create_address_object('web_server_02', ...) →")
print("   create_address_object('web_server_03', ...) →")
print("   create_address_object('web_server_04', ...) →")
print("   create_address_object('web_server_05', ...) →")
print("   create_address_group('new_web_servers', 'web_server_02,web_server_03,...')")

### Visualizing the Tool Calling Flow

Let's create a visualization that shows exactly how the ReAct loop works with tool calls. This helps understand the Reason → Act → Observe cycle:

In [ ]:
def visualize_tool_calling_trace(stream_output):
    """
    Create a visual trace of the tool calling sequence.
    
    This helps understand:
    - What tools were called
    - In what order
    - What decisions the agent made
    - How results flowed back
    """
    print("\n" + "="*80)
    print("TOOL CALLING TRACE VISUALIZATION")
    print("="*80)
    
    step_num = 1
    
    for event in stream_output:
        for node_name, data in event.items():
            if node_name == "agent":
                print(f"\n[Step {step_num}] 🤖 AGENT NODE - Reasoning Phase")
                print("-" * 80)
                
                if "messages" in data:
                    for msg in data["messages"]:
                        if hasattr(msg, "tool_calls") and msg.tool_calls:
                            print(f"  Decision: Call {len(msg.tool_calls)} tool(s)")
                            for tc in msg.tool_calls:
                                print(f"    → Tool: {tc['name']}")
                                print(f"      Args: {tc['args']}")
                            print(f"  Route: → tools node")
                        else:
                            if hasattr(msg, "content") and msg.content:
                                content_preview = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
                                print(f"  Decision: No more tools needed")
                                print(f"  Response: {content_preview}")
                                print(f"  Route: → END")
                
                step_num += 1
                
            elif node_name == "tools":
                print(f"\n[Step {step_num}] 🔧 TOOLS NODE - Acting Phase")
                print("-" * 80)
                
                if "messages" in data:
                    for msg in data["messages"]:
                        if hasattr(msg, "content"):
                            tool_name = msg.name if hasattr(msg, "name") else "unknown_tool"
                            result_preview = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
                            print(f"  Executed: {tool_name}")
                            print(f"  Result: {result_preview}")
                print(f"  Route: → agent node (loop back)")
                
                step_num += 1
    
    print("\n" + "="*80)
    print("END OF TRACE")
    print("="*80)

# Test with a simple query
print("Let's visualize a simple tool calling sequence:")
print("\nQuery: List all address objects in Texas folder\n")

stream = app.stream({"messages": [HumanMessage(content="List all address objects in Texas folder")]})
visualize_tool_calling_trace(stream)

print("\n💡 Key observations from the trace:")
print("   1. Agent always reasons before acting")
print("   2. Tools execute and return results")
print("   3. Control loops back to agent after each tool call")
print("   4. Agent decides when to stop (no more tool_calls)")
print("   5. This creates the Reason → Act → Observe → Reason cycle")

### Tool Error Handling and Recovery

One of the most powerful aspects of the ReAct pattern is the agent's ability to handle errors gracefully. Let's see how the agent responds when tools return error messages:

**Scenarios to test:**
1. **Duplicate object creation** - What happens if we try to create an object that already exists?
2. **Missing prerequisites** - What if we try to create a group with non-existent members?
3. **Invalid input** - What if we provide malformed data?

The agent should be able to:
- Understand error messages
- Explain the problem to the user
- Suggest corrective actions
- Potentially retry with corrected parameters

In [ ]:
# Test 4: Error handling - Try to create a duplicate object
print("\nTEST 4: Error Handling - Duplicate Object Creation")
print("="*60)

query = "Create an address object called web_server_01 at 10.0.1.10/32 in Texas folder"
stream = app.stream({"messages": [HumanMessage(content=query)]})

print_stream(stream)

print("\n" + "="*60)
print("✅ INTELLIGENT ERROR HANDLING!")
print("   1. Agent called create_address_object")
print("   2. Tool returned error: object already exists")
print("   3. Agent understood the error")
print("   4. Agent explained the problem to the user")
print("   5. Agent suggested using fetch_address_object instead")
print("\n💡 The agent can read and understand tool errors!")

In [ ]:
# Test 5: Error handling - Missing prerequisites
print("\nTEST 5: Error Handling - Creating Group with Non-Existent Members")
print("="*60)

query = "Create an address group called test_group with members: server_a, server_b, server_c in Texas folder"
stream = app.stream({"messages": [HumanMessage(content=query)]})

print_stream(stream)

print("\n" + "="*60)
print("✅ PREREQUISITE VALIDATION!")
print("   1. Agent called create_address_group")
print("   2. Tool validated members don't exist")
print("   3. Tool returned error with missing member names")
print("   4. Agent explained the prerequisite failure")
print("   5. Agent suggested creating the address objects first")
print("\n💡 Tools can perform validation and return helpful error messages!")

In [ ]:
# Test 6: Error handling with recovery - Invalid input
print("\nTEST 6: Error Handling - Invalid IP Format")
print("="*60)

query = "Create an address object called bad_ip at 10.0.1 in Texas folder"
stream = app.stream({"messages": [HumanMessage(content=query)]})

print_stream(stream)

print("\n" + "="*60)
print("✅ INPUT VALIDATION!")
print("   1. Agent called create_address_object with malformed IP")
print("   2. Tool validated the IP/netmask format")
print("   3. Tool returned error with format requirements")
print("   4. Agent explained the correct format to the user")
print("\n💡 Best practices for tool design:")
print("   - Validate inputs before processing")
print("   - Return clear, actionable error messages")
print("   - Include examples of valid formats")
print("   - Help the agent help the user")

### What We Covered

1. **StructuredTool Pattern for Notebooks**
   - Why `StructuredTool.from_function()` instead of `@tool` decorator
   - Function prefix convention (use `_` for internal functions)
   - Explicit name and description control
   - No re-execution conflicts

2. **Comprehensive SCM Tool Suite**
   - `fetch_address_object` - Retrieve existing objects
   - `create_address_object` - Create with validation
   - `list_addresses` - List with filtering
   - `create_address_group` - Group management
   - Real pan-scm-sdk API patterns

3. **The add_messages Reducer**
   - Automatic state management
   - No more manual `.append()`
   - Cleaner, simpler code
   - `Annotated[Sequence[BaseMessage], add_messages]` pattern

4. **Message Types**
   - `HumanMessage` - User input
   - `AIMessage` - LLM responses with tool_calls
   - `SystemMessage` - LLM instructions
   - `ToolMessage` - Tool execution results
   - `BaseMessage` - Parent class for all

5. **The ReAct Pattern**
   - **Reason**: Agent analyzes situation and decides on tools
   - **Act**: Tools execute actions via ToolNode
   - **Observe**: Agent receives results via ToolMessage
   - **Loop**: Continues until agent decides to stop
   - Autonomous multi-step workflows

6. **Graph Components**
   - Agent node with tool-bound LLM
   - ToolNode handling multiple tools
   - `should_continue` conditional routing
   - Loop edge from tools → agent

7. **Error Handling**
   - Input validation in tools
   - Clear error messages
   - Agent understanding errors
   - Suggesting corrective actions
   - Prerequisite validation

8. **Tool Calling Visualization**
   - Trace tool execution sequences
   - Understanding decision flow
   - Debugging tool chains

### Evolution of Our Agents

| Notebook | Pattern | State Management | Tool Pattern | Capabilities |
|----------|---------|------------------|--------------|--------------|
| 104 | Simple Bot | Manual append | None | Just talks |
| 105 | Memory Bot | Manual sync | None | Remembers context |
| **110** | **ReAct Agent** | **Reducer!** | **StructuredTool** | **Takes actions!** |

### Key Insights

**The StructuredTool Pattern:**
```python
# Define function with _ prefix
def _my_tool(param: str) -> str:
    """Comprehensive docstring for LLM."""
    return result

# Create tool explicitly
my_tool = StructuredTool.from_function(
    func=_my_tool,
    name="my_tool",
    description="Clear description for LLM decision making"
)
```

**Why This Matters:**
- No decorator conflicts on cell re-execution
- Explicit control over tool metadata
- Easier debugging in notebooks
- Better for iterative development

**The Reducer Revolution:**
```python
# Before (Notebook 109)
state["messages"].append(AIMessage(content=response.content))
conversation_history = result["messages"]

# After (Notebook 110)
messages: Annotated[Sequence[BaseMessage], add_messages]
# That's it! The reducer handles everything!
```

**Tool Decision Making:**
The LLM uses tool descriptions to:
1. Understand what each tool does
2. Decide which tool to call
3. Determine what parameters to pass
4. Know when no tool is needed
5. Handle tool errors intelligently

**The ReAct Loop:**
```
User Query → Agent (Reason) → Tool Call Decision → ToolNode (Act) → 
ToolMessage (Observe) → Agent (Reason) → More Tools OR Final Answer → END
```

### Real-World SCM Applications

This pattern enables:

1. **Autonomous Object Management**
   - Create multiple address objects in sequence
   - Validate existence before creation
   - Handle name conflicts gracefully

2. **Intelligent Grouping**
   - List objects by tags
   - Create groups from filtered results
   - Validate prerequisites automatically

3. **Multi-Step Workflows**
   - "Create 4 servers and group them" → 5 tool calls
   - "List production servers and create security rule" → 3+ tool calls
   - Agent handles sequencing automatically

4. **Error Recovery**
   - Detects duplicate objects
   - Validates input formats
   - Suggests corrective actions
   - Explains errors clearly

### Common Patterns You've Mastered

1. **Tool Creation with StructuredTool**
   ```python
   def _tool_function(param: str) -> str:
       """Docstring is critical!"""
       return result
   
   tool = StructuredTool.from_function(
       func=_tool_function,
       name="tool_name",
       description="What the tool does and when to use it"
   )
   ```

2. **State with Reducer**
   ```python
   messages: Annotated[Sequence[BaseMessage], add_messages]
   ```

3. **Tool Binding**
   ```python
   tools = [tool1, tool2, tool3]
   model_with_tools = llm.bind_tools(tools)
   ```

4. **Conditional Routing**
   ```python
   def should_continue(state):
       if last_message.tool_calls:
           return "tools"
       return END
   ```

5. **ToolNode Integration**
   ```python
   tool_node = ToolNode(tools)
   graph.add_node("tools", tool_node)
   ```

6. **Loop Creation**
   ```python
   graph.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
   graph.add_edge("tools", "agent")  # Loop back!
   ```

### Best Practices for Tool Design

✅ **DO:**
- Use `StructuredTool.from_function()` in notebooks
- Prefix internal functions with `_`
- Write comprehensive docstrings
- Validate inputs before processing
- Return clear, structured responses
- Include error context and suggestions
- Simulate real API patterns

❌ **DON'T:**
- Use `@tool` decorator in notebooks (causes conflicts)
- Return generic error messages
- Skip input validation
- Assume prerequisites exist
- Return just "success" or "error"

### Performance Considerations

**Tool Calling Overhead:**
- Each tool call requires LLM invocation
- Complex workflows may take time
- Consider batching where possible
- Monitor token usage

**Agent Decision Quality:**
- Clear tool descriptions improve accuracy
- Good error messages help agent recovery
- System prompts guide behavior
- Tool naming matters

### Next Steps

**Ready to continue?** Upcoming notebooks will cover:
- Notebook 111: Advanced ReAct patterns and plan-and-execute
- Checkpointing and conversation persistence
- Multi-agent systems
- Streaming responses
- Human-in-the-loop patterns

**Want more practice?** Try the exercises in Section 10:
1. Security policy tools (intermediate)
2. Update address object tool (intermediate)
3. Complex multi-tool workflows (advanced)
4. Retry logic implementation (advanced)
5. Tool execution dashboard (advanced)

### Key Takeaways

✅ **StructuredTool is better for notebooks** - No decorator conflicts!
✅ **Reducers automate state management** - No more manual appending!
✅ **ReAct pattern = Reasoning + Acting** - Most powerful agent pattern
✅ **Tool descriptions are critical** - The LLM relies on them
✅ **Conditional edges create loops** - Enable multi-step workflows
✅ **ToolNode handles all tools** - Just pass the list!
✅ **Error handling is essential** - Tools should validate and explain
✅ **SCM patterns are production-ready** - Real API simulation

### Architecture Review

**What We Built:**

```
┌──────────────────────────────────────────────────────────┐
│                     ReAct Agent                          │
│                                                          │
│  ┌────────┐     ┌──────────────┐     ┌──────────────┐ │
│  │ START  │────→│    Agent     │────→│     END      │ │
│  └────────┘     │  (Reasoning) │     └──────────────┘ │
│                 └──────┬───────┘                       │
│                        │ ↑                             │
│                        ↓ │                             │
│                 ┌──────────────┐                       │
│                 │   ToolNode   │                       │
│                 │   (Acting)   │                       │
│                 │              │                       │
│                 │ • fetch      │                       │
│                 │ • create     │                       │
│                 │ • list       │                       │
│                 │ • group      │                       │
│                 └──────────────┘                       │
└──────────────────────────────────────────────────────────┘
```

**Message Flow:**
1. HumanMessage → Agent
2. Agent → AIMessage (with tool_calls)
3. ToolNode → ToolMessage (results)
4. Agent → AIMessage (final response)
5. END

### The Power You've Gained

You can now build agents that:
- **Think** before acting (reasoning)
- **Use tools** to gather information and take actions
- **Chain operations** together autonomously
- **Handle errors** gracefully
- **Validate inputs** and prerequisites
- **Explain** what they're doing
- **Recover** from failures

This is **production-grade AI agent architecture** for Strata Cloud Manager automation!

### Comparison with Manual Approaches

**Without ReAct Agent:**
```python
# Manual approach - YOU write the logic
user_query = "Create 4 servers and group them"
# Parse query manually
# Call API 4 times for servers
for server in servers:
    response = client.address.create(server_config)
    if response.error:
        handle_error()
# Then call API for group
response = client.address_group.create(group_config)
```

**With ReAct Agent:**
```python
# Agent approach - AGENT figures it out
user_query = "Create 4 servers and group them"
response = app.stream({"messages": [HumanMessage(content=user_query)]})
# Agent autonomously:
# - Understands intent
# - Creates 4 servers sequentially
# - Validates each creation
# - Creates group with correct members
# - Handles any errors
# - Explains results
```

**The agent writes the logic FOR you!**

### Workshop Integration

This notebook builds on:
- **101-103**: Graph fundamentals
- **104-105**: State management patterns  
- **106-107**: Advanced routing
- **108**: LLM integration
- **109**: Conversational memory

And prepares you for:
- **111**: Advanced agent patterns
- **Checkpointing**: Persistence
- **Multi-agent**: Agent collaboration
- **Production**: Real SCM automation

Excellent work! You're now ready to build sophisticated AI agents for real-world Strata Cloud Manager automation! 🚀

### Additional Resources

- **pan-scm-sdk Documentation**: https://pan.dev/scm/sdk/
- **LangGraph Documentation**: https://langchain-ai.github.io/langgraph/
- **ReAct Paper**: https://arxiv.org/abs/2210.03629
- **Workshop Repository**: Check `/docs/examples/` for more SCM patterns